In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/associative-rules-data/retail_transactional.csv
/kaggle/input/associative-rules-data/groceries.csv


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [3]:
import os
from collections import Counter

# File paths
GROCERIES_PATH = '/kaggle/input/associative-rules-data/groceries.csv'
RETAIL_PATH = '/kaggle/input/associative-rules-data/retail_transactional.csv'

def inspect_and_analyze(file_path, name):
    print(f"\n{'='*50}")
    print(f" Analyzing Dataset: {name}")
    print(f"{'='*50}")
    
    if not os.path.exists(file_path):
        print(f" File not found: {file_path}")
        return

    # 1. Preview raw content (First 5 lines) to understand structure
    print(" Raw File Content (First 5 lines):")
    try:
        with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
            raw_lines = [f.readline().strip() for _ in range(5)]
            for i, line in enumerate(raw_lines):
                print(f"  Line {i+1}: {line}")
    except Exception as e:
        print(f"  Could not read raw lines: {e}")
        return

    # 2. Process file line by line to calculate statistics
    transactions = []
    unique_items = set()
    
    try:
        with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
            for line in f:
                line = line.strip()
                if not line: continue
                
                # Split by comma (assuming standard CSV or basket format)
                # We remove quotes to clean up data like "milk" -> milk
                items = [i.strip().replace('"', '') for i in line.split(',')]
                
                # Filter out empty strings
                items = [i for i in items if i]
                
                if items:
                    transactions.append(items)
                    unique_items.update(items)

        # 3. Calculate Statistics
        num_transactions = len(transactions)
        num_unique_items = len(unique_items)
        
        if num_transactions == 0:
            print(" Dataset appears empty.")
            return

        avg_len = sum(len(t) for t in transactions) / num_transactions
        
        # Calculate Top 5 Frequent Items
        all_items = [item for t in transactions for item in t]
        item_counts = Counter(all_items)
        top_5 = item_counts.most_common(5)

        print(f"\n Statistics Summary:")
        print(f"  • Total Transactions (Rows): {num_transactions:,}")
        print(f"  • Unique Items Found: {num_unique_items:,}")
        print(f"  • Avg Items per Row: {avg_len:.2f}")
        
        print(f"\n Top 5 Frequent Items (Check for headers or strange IDs):")
        for item, count in top_5:
            support = count / num_transactions
            print(f"  - '{item}': {count} times (Freq: {support:.4f})")

    except Excep as e:
        print(f" Error calculating stats: {e}")

# Run analysis
inspect_and_analyze(GROCERIES_PATH, "Groceries")
inspect_and_analyze(RETAIL_PATH, "Retail Transactional")



 Analyzing Dataset: Groceries
 Raw File Content (First 5 lines):
  Line 1: citrus fruit,semi-finished bread,margarine,ready soups
  Line 2: tropical fruit,yogurt,coffee
  Line 3: whole milk
  Line 4: pip fruit,yogurt,cream cheese ,meat spreads
  Line 5: other vegetables,whole milk,condensed milk,long life bakery product

 Statistics Summary:
  • Total Transactions (Rows): 9,835
  • Unique Items Found: 169
  • Avg Items per Row: 4.41

 Top 5 Frequent Items (Check for headers or strange IDs):
  - 'whole milk': 2513 times (Freq: 0.2555)
  - 'other vegetables': 1903 times (Freq: 0.1935)
  - 'rolls/buns': 1809 times (Freq: 0.1839)
  - 'soda': 1715 times (Freq: 0.1744)
  - 'yogurt': 1372 times (Freq: 0.1395)

 Analyzing Dataset: Retail Transactional
 Raw File Content (First 5 lines):
  Line 1: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
  Line 2: 30,31,32
  Line 3: 33,34,35
  Line 4: 36,37,38,39,40,41,42,43,44,45,46
  Line 5: 38,39,47,48

 Statistics Summ